In [10]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import torch

In [12]:
# Import necessary libraries
import pandas as pd

# Load your labeled dataset (custom CSV file)
labeled_data = pd.read_csv('../data/labeled_data.csv', encoding='utf-8')

# Rename the existing columns for clarity
labeled_data.columns = ['Labeled Data', 'ner_tags']

# Display first few rows of the dataset
print(labeled_data.head())



  Labeled Data ner_tags
0     ተገጣጣሚዎቹን        O
1        የእንጨት        O
2         ብሎኮች        O
3        ከመነሻዬ        O
4         ሲሸምቱ        O


In [16]:
# Group tokens and labels by sentence
grouped_data = labeled_data.groupby('sentence_id').agg({'token': list, 'label': list}).reset_index()

# Convert the DataFrame into a Hugging Face Dataset object
dataset = Dataset.from_pandas(grouped_data)


KeyError: 'sentence_id'

In [19]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import torch

# Step 1: Load Your Labeled Dataset
labeled_data = pd.read_csv('../data/labeled_data.csv', encoding='utf-8')

# Step 2: Create a sentence ID to group tokens by sentence
# Add a new column 'sentence_id' for grouping sentences
# labeled_data['sentence_id'] = labeled_data.index // 10  # Adjust based on your sentence lengt

# # Rename the existing columns for clarity
# labeled_data.columns = ['Labeled Data', 'ner_tags']

# # Step 4: Group tokens and labels by sentence
# grouped_data = labeled_data.groupby('sentence_id').agg({'token': list, 'label': list}).reset_index()

# Step 5: Convert the DataFrame into a Hugging Face Dataset object
dataset = Dataset.from_pandas(labeled_data)

# Step 6: Load Pre-trained Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Step 7: Tokenization and Label Alignment Function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(int(label[word_idx]))  # Assign label to first token
            else:
                label_ids.append(-100)  # Assign -100 to other tokens of the same word
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Step 8: Apply the Tokenization
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Step 9: Load the Pre-trained Model for Token Classification
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=9)  # Adjust num_labels

# Step 10: Define Evaluation Metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    return metric.compute(predictions=true_predictions, references=true_labels)

# Step 11: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay
)

# Step 12: Fine-Tune the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Assuming a single dataset for training
    eval_dataset=tokenized_datasets,  # Using the same for evaluation; modify if needed
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Step 13: Evaluate the Model
trainer.evaluate()

# Step 14: Inference Example
sentence = "አቶ ሳሙኤል አያሌው ኢትዮጵያ የመንግሥት ተቋም ነው።"
inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=2)
print(predictions)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/40931 [00:00<?, ? examples/s]

KeyError: 'token'

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric
import numpy as np
import torch

# Step 1: Load Your Labeled Dataset
labeled_data = pd.read_csv('../data/labeled_data.csv', encoding='utf-8')

# Step 2: Create a sentence ID to group tokens by sentence
labeled_data['sentence_id'] = labeled_data.index // 10  # Adjust based on your sentence length

# Step 3: Group tokens and labels by sentence
grouped_data = labeled_data.groupby('sentence_id').agg({'Labeled Data': list, 'ner_tags': list}).reset_index()

# Verify the grouped data structure
print(grouped_data.head())

# Step 4: Convert the DataFrame into a Hugging Face Dataset object
dataset = Dataset.from_pandas(grouped_data)

# Check the dataset columns
print(dataset.column_names)

# Step 5: Load Pre-trained Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Step 6: Define Label Mapping
label_names = list(set(labeled_data['ner_tags']))  # Extract unique labels from your dataset
label_map = {label: idx for idx, label in enumerate(label_names)}  # Create a mapping of label to index

# Step 7: Tokenization and Label Alignment Function
def tokenize_and_align_labels(examples):
    # Accessing the columns directly to ensure they are correctly named
    tokenized_inputs = tokenizer(examples['Labeled Data'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):  # Use ner_tags here
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label_map[label[word_idx]])  # Convert label to its corresponding integer
            else:
                label_ids.append(-100)  # Assign -100 to other tokens of the same word
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels  # Store labels in 'labels'
    return tokenized_inputs

# Step 8: Apply the Tokenization
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Step 9: Load the Pre-trained Model for Token Classification
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_names))  # Use the dynamic label count

# Step 10: Define Evaluation Metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    return metric.compute(predictions=true_predictions, references=true_labels)

# Step 11: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay
)

# Step 12: Fine-Tune the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Assuming a single dataset for training
    eval_dataset=tokenized_datasets,  # Using the same for evaluation; modify if needed
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Step 13: Evaluate the Model
trainer.evaluate()

# Step 14: Inference Example
sentence = "አቶ ሳሙኤል አያሌው ኢትዮጵያ የመንግሥት ተቋም ነው።"
inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=2)
print(predictions)
